In [2]:
!pip install lightgbm

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 56.9 MB/s eta 0:00:00

[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip


In [3]:
!pip install imblearn

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip


In [9]:
import os
import pickle
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, classification_report, f1_score, precision_score, recall_score
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from tqdm import tqdm  # For progress bars

# Helper function to calculate top-3 accuracy
def calculate_top3_accuracy(y_true, y_probs):
    top_3_preds = np.argsort(y_probs, axis=1)[:, -3:]
    matches = [y_true[i] in top_3_preds[i] for i in range(len(y_true))]
    return np.mean(matches)

# Helper function to calculate Threshold Accuracy
def calculate_threshold_accuracy(y_true, y_pred, threshold=1):
    return np.mean(np.abs(y_true - y_pred) <= threshold)

# Helper function to evaluate metrics
def evaluate_metrics(y_true, y_pred, y_probs=None, threshold=1):
    metrics = {
        'accuracy': accuracy_score(y_true, y_pred),
        'precision': precision_score(y_true, y_pred, average='weighted', zero_division=0),
        'recall': recall_score(y_true, y_pred, average='weighted', zero_division=0),
        'f1_score': f1_score(y_true, y_pred, average='weighted'),
        'threshold_accuracy': calculate_threshold_accuracy(y_true, y_pred, threshold),  # Added Threshold Accuracy
        'classification_report': classification_report(y_true, y_pred, zero_division=0)
    }
    if y_probs is not None:
        metrics['top_3_accuracy'] = calculate_top3_accuracy(y_true, y_probs)
    return metrics


# Helper function to load dataset
def load_data(file_path):
    print(f"Loading data from: {file_path}")
    data = pd.read_csv(file_path)
    label_columns = [col for col in data.columns if "slider" in col]
    feature_columns = [col for col in data.columns if col not in label_columns + ["Timestamp", "participant_id"]]
    X = data[feature_columns]
    return X, label_columns, data

# Function to train models per label with class imbalance handling
from sklearn.utils.class_weight import compute_class_weight
from imblearn.over_sampling import SMOTE
import numpy as np

# Function to train models per label with class imbalance handling
def train_model_per_label(X_train, y_train, X_test, y_test, model_save_path, label_name, model_type="xgb"):
    os.makedirs(model_save_path, exist_ok=True)

    # Convert labels to integers to prevent XGBoost issues
    y_train = y_train.astype(int)
    y_test = y_test.astype(int)

    # Ensure X_train is a NumPy array (to avoid KeyError when using vstack)
    X_train = X_train.to_numpy()
    X_test = X_test.to_numpy()

    # Ensure all possible classes are present
    all_classes = np.arange(11)  # Since valid labels are [0-10]
    unique_classes = np.unique(y_train)

    # Check for missing classes and add dummy samples to prevent errors
    missing_classes = set(all_classes) - set(unique_classes)
    if missing_classes:
        print(f"Warning: Missing classes {missing_classes} in training data. Adding dummy samples.")
        for cls in missing_classes:
            X_train = np.vstack([X_train, X_train[0]])  # Duplicate first row
            y_train = np.append(y_train, cls)  # Append missing class label

    # Compute class weights for imbalance handling
    class_weights = compute_class_weight("balanced", classes=all_classes, y=y_train)
    class_weight_dict = {c: w for c, w in zip(all_classes, class_weights)}

    # **🔹 Fix: Apply SMOTE only if the smallest class has at least 6 samples**
    min_class_samples = min(np.bincount(y_train))
    if len(unique_classes) > 2 and min_class_samples >= 6:
        smote = SMOTE(sampling_strategy="auto", random_state=42)
        X_train, y_train = smote.fit_resample(X_train, y_train)
        print(f"Applied SMOTE for label {label_name}")
    else:
        print(f"Skipping SMOTE for {label_name} (min class has {min_class_samples} samples)")

    # Configure model parameters
    num_classes = len(all_classes)
    if model_type == "xgb":
        model = XGBClassifier(
            objective="multi:softmax",
            num_class=num_classes,
            eval_metric="mlogloss",
            use_label_encoder=False,
            scale_pos_weight=class_weights
        )
    elif model_type == "lgbm":
        model = LGBMClassifier(
            objective="multiclass",
            num_class=num_classes,
            is_unbalance=True,
            num_leaves=15,
            min_data_in_leaf=5,
            max_depth=6,
            learning_rate=0.05,
            feature_fraction=0.8,
            bagging_fraction=0.8,
            bagging_freq=5,
            lambda_l1=0.5,
            lambda_l2=0.5
        )
    else:
        raise ValueError("Unsupported model type")

    # Train model
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_probs = model.predict_proba(X_test) if hasattr(model, "predict_proba") else None

    # Save model
    with open(os.path.join(model_save_path, f"{model_type}_{label_name}.pkl"), "wb") as f:
        pickle.dump(model, f)

    return evaluate_metrics(y_test, y_pred, y_probs)


# Main training loop
def train_all_models(data_dir, root_dir):
    results = []
    folds = sorted(set(f.split("_")[1] for f in os.listdir(data_dir) if f.endswith(".csv")))

    for fold in tqdm(folds, desc="Processing Folds"):
        train_file = os.path.join(data_dir, f"fold_{fold}_train.csv")
        test_file = os.path.join(data_dir, f"fold_{fold}_test.csv")
        
        if not os.path.exists(train_file) or not os.path.exists(test_file):
            print(f"Skipping fold {fold}: Missing train or test data")
            continue
        
        X_train, label_columns, train_data = load_data(train_file)
        X_test, _, test_data = load_data(test_file)
        
        for label_column in label_columns:
            y_train, y_test = train_data[label_column], test_data[label_column]
            fold_save_dir = os.path.join(root_dir, f"fold_{fold}", "models", label_column)
            os.makedirs(fold_save_dir, exist_ok=True)
            
            for model_type in ["xgb", "lgbm"]:
                model_results = train_model_per_label(X_train, y_train, X_test, y_test, os.path.join(fold_save_dir, model_type), label_column, model_type)
                results.append({"Fold": fold, "Label": label_column, "Model": model_type.upper(), **model_results})
    
    results_df = pd.DataFrame(results)
    results_df.to_csv("fold__LGBM_XGB_model_performance_results.csv", index=False)
    print("Model training results saved to fold__LGBM_XGB_model_performance_results.csv")

if __name__ == "__main__":
    data_dir = "folds"
    root_dir = "folds_original_LGBM_XGB_models"
    train_all_models(data_dir, root_dir)


Processing Folds:   0%|          | 0/5 [00:00<?, ?it/s]

Loading data from: folds/fold_1_train.csv
Loading data from: folds/fold_1_test.csv
Applied SMOTE for label P1 (sliderNeutralPos)


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [11:20:53] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "scale_pos_weight" } are not used.

  warnings.warn(smsg, UserWarning)


Applied SMOTE for label P1 (sliderNeutralPos)
[LightGBM] [Warning] min_data_in_leaf is set=5, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=5
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] lambda_l2 is set=0.5, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.5
[LightGBM] [Warning] lambda_l1 is set=0.5, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.5
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] min_data_in_leaf is set=5, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=5
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] lambda_l2 is 

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


...

[LightGBM] [Warning] min_data_in_leaf is set=5, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=5
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] lambda_l2 is set=0.5, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.5
[LightGBM] [Warning] lambda_l1 is set=0.5, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.5
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] min_data_in_leaf is set=5, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=5
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] lambda_l2 is set=0.5, reg_lambda=0.0 will be ignored. Curre

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [11:34:57] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "scale_pos_weight" } are not used.

  warnings.warn(smsg, UserWarning)


Skipping SMOTE for PRO3 (sliderNeutralPos) (min class has 1 samples)
[LightGBM] [Warning] min_data_in_leaf is set=5, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=5
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] lambda_l2 is set=0.5, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.5
[LightGBM] [Warning] lambda_l1 is set=0.5, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.5
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] min_data_in_leaf is set=5, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=5
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] 

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
Processing Folds: 100%|██████████| 5/5 [14:10<00:00, 170.10s/it]

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] min_data_in_leaf is set=5, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=5
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[Light